In [16]:
import dotenv
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [17]:
# assert version is > 1.0.0
assert openai.__version__ > "1.0.0", "Please upgrade the OpenAI Python client: pip install -U openai"
print(openai.__version__)

1.3.6


In [18]:
pip install -U openai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Files for context of the assistant

In [19]:
# import glob
# # get all python files inside data/tinygrad even inside subfolders etc
# filepath = "data/tinygrad/"
# python_files = glob.glob(filepath + "**/*.py", recursive=True)
# print(f"Found {len(python_files)} python files")

# def is_usefull_python_file(python_file_path):
#     # just to remove useless file for the assistant 
#     bad_strings_to_not_match = ["test", "mlperf", "setup.py",
#                                 "runtime", "examples", "renderer",
#                                  "openpilot", "models","adreno", 
#                                  "extra", "jit", "sz.py"]
#     if "resnet.py" in python_file_path:
#         return True
#     for bad_string in bad_strings_to_not_match:
#         if bad_string in python_file_path:
#             return False
#     return True
# # print files with model in the path

# print("Files with model in the path")
# print(list(filter(lambda x: "model" in x, python_files)))


# python_files = list(filter(is_usefull_python_file, python_files))
# print(f"Found {len(python_files)} python files after filtering")
# assert len(python_files) <= 20, "Openai limit is 20 files"
# print("\n".join(python_files))

In [20]:
import glob
# get all python files inside data/tinygrad even inside subfolders etc
filepath = "data/"
python_files = glob.glob(filepath + "**/*.csv", recursive=True)
print(f"Found {len(python_files)} python files")

def is_usefull_python_file(python_file_path):
    # just to remove useless file for the assistant 
    bad_strings_to_not_match = ["test", "mlperf", "setup.py",
                                "runtime", "examples", "renderer",
                                 "openpilot", "models","adreno", 
                                 "extra", "jit", "sz.py"]
    if "resnet.py" in python_file_path:
        return True
    for bad_string in bad_strings_to_not_match:
        if bad_string in python_file_path:
            return False
    return True
# print files with model in the path

print("Files with model in the path")
print(list(filter(lambda x: "model" in x, python_files)))


python_files = list(filter(is_usefull_python_file, python_files))
print(f"Found {len(python_files)} python files after filtering")
assert len(python_files) <= 20, "Openai limit is 20 files"
print("\n".join(python_files))

Found 1 python files
Files with model in the path
[]
Found 1 python files after filtering
data\qa1-simp.csv


## Upload the files to Openai

In [21]:
from openai import OpenAI
from tqdm.auto import tqdm
client = OpenAI()
uploaded_files = []
for filename in tqdm(python_files):
    file_obj = client.files.create(
        file=open(filename, 'rb'),
        purpose='assistants',
        
    )
    uploaded_files.append(file_obj.id)

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
uploaded_files

['file-JJ99FhhmXersv1NO2pfIicHH']

## Create the assistant

In [23]:
tinygrad_assistant = openai.beta.assistants.create(
    instructions="You are helping coders understand the tinygrad library. You have access to the source code of the codebase. You are a helpful tinygrad assistant.",
    name="Tinygrad Assistant (API)",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=uploaded_files,
)

## Create an empty thread
Empty threads are not linked to an assistant yet. Multiple assistants can be used inside the same thread :)

In [24]:
thread = openai.beta.threads.create()
thread_id = thread.id

In [25]:
query = "Q가 말한 것은 몇가지야?"

In [26]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=query,
)
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=tinygrad_assistant.id)

import time
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run.completed_at:
        elapsed = run.completed_at - run.created_at
        elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        print(f"Run completed in {elapsed}")
        break
    print("Waiting 1sec")
    time.sleep(1)

Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Run completed in 00:00:23


In [27]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
last_message = messages.data[0]

text = last_message.content[0].text.value
print(text)

Q가 말한 것은 총 7가지입니다. 다음은 낮은 자존감으로 인해 나타나는 영향들입니다:

1. 일이 잘못되면 스스로를 낮추거나 의심하거나 비난한다.
2. 일이 잘 풀리지 않을 거라 생각하고 도전이나 기회를 피한다.
3. 다른 사람들보다 능력이 부족하다고 믿고 적게 성취한다.
4. 비판이나 거절에 화를 내거나 괴로워한다.
5. 다른 사람을 기쁘게 하기 위해 상대에게 맞추거나 자신을 깎아내린다.
6. 여가 활동을 많이 하지 않고 판단/평가 받을 수 있는 활동을 피한다.
7. 자기 관리에 영향을 받아 소중히 여기지 않고, 외모 관리에 소홀하거나 그 반대로 완벽해 보이려 애쓴다.
